In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
import datetime
import os
import pickle

# grid
根据地理信息划分格子，统计格子上历史平均时间

In [ ]:
dir = "./history/"
history = pd.DataFrame()
for root, dirs, files in os.walk(dir):
    for file in tqdm(files):
        s = os.path.join(root,file)
        print (s)
        line = file.split('_')[0]
        with open(s, 'rb') as data_file:
            h = pickle.load(data_file).reset_index()
        h['Line'] = line
        h['is_rain'] = file.split('_')[1]
        h['is_workday'] = file.split('_')[2]
        h['is_peek'] = file.split('_')[3].split('.')[0]

        history = pd.concat([history,h])
with open('./data/full_history', 'wb') as data_file:
    pickle.dump(history, data_file)
print('finished')

In [ ]:
from tqdm import tqdm, tqdm_pandas
import math

tqdm.pandas()


def read_station_loc(line, on, start):
    path = './station/' + str(line) + '.txt'
    if os.path.exists(path):
        bus1 = pd.read_table(path, header=None, sep=',', encoding='utf-8')
        bus1.columns = ["O_UP", "Source_Station", "O_LATITUDE", "O_LONGITUDE"]
        b = bus1[(bus1['O_UP'] == on) & (bus1['Source_Station'] == start)]
        if b.shape[0] > 0:
            return str([b.iloc[0]['O_LATITUDE'], b.iloc[0]['O_LONGITUDE']])
        else:
            return str([-1, -1])
    else:
        return str([-1, -1])

#38.65936 40.07771
#116.917 117.97901
max_x = 40.0777#max(test2['s_x'].max(),test2['e_x'].max()) + 0.00001
min_x = 38.6325#min(test2['s_x'].min(),test2['e_x'].min())
max_y = 118.16#max(test2['s_y'].max(),test2['e_y'].max()) + 0.00001
min_y = 116.900#min(test2['s_y'].min(),test2['e_y'].min())
lx = (max_x - min_x) / 50
ly = (max_y - min_y) / 33
print(min_x, max_x)
print(min_y, max_y)
print(lx, ly)


def get_ij(x, y):
    i = math.floor((x - min_x) / lx)
    j = math.floor((y - min_y) / ly)
    num = i * 33 + (j + 1)
    return num


# progress_apply
def process(train_id):
    train_id['Source_loc'] = train_id.progress_apply(lambda x: read_station_loc(x['Line'], x['O_UP'], x['Source_Station']), axis=1)
    train_id['end_loc'] = train_id.progress_apply(lambda x: read_station_loc(x['Line'], x['O_UP'], x['Target_Station']), axis=1)
    train_id['s_x'] = train_id.Source_loc.apply(lambda x: x[1:-1].split(',')[0].replace(' ', ""))
    train_id['s_y'] = train_id.Source_loc.apply(lambda x: x[1:-1].split(',')[1].replace(' ', ""))
    train_id['e_x'] = train_id.end_loc.apply(lambda x: x[1:-1].split(',')[0].replace(' ', ""))
    train_id['e_y'] = train_id.end_loc.apply(lambda x: x[1:-1].split(',')[1].replace(' ', ""))

    train_id['s_x'] = train_id.s_x.str.extract('(\d+\.\d+)', expand=False)
    train_id['s_y'] = train_id.s_y.str.extract('(\d+\.\d+)', expand=False)
    train_id['e_x'] = train_id.e_x.str.extract('(\d+\.\d+)', expand=False)
    train_id['e_y'] = train_id.e_y.str.extract('(\d+\.\d+)', expand=False)

    train_id['s_x'] = train_id.s_x.astype(np.float32)
    train_id['s_y'] = train_id.s_y.astype(np.float32)
    train_id['e_x'] = train_id.e_x.astype(np.float32)
    train_id['e_y'] = train_id.e_y.astype(np.float32)

    train_id['s_ij'] = train_id.progress_apply(
        lambda x: get_ij(x['s_x'], x['s_y']) if (np.all(pd.notnull(x['s_x'])) and np.all(pd.notnull(x['s_y']))) else -1,
        axis=1)
    train_id['e_ij'] = train_id.progress_apply(
        lambda x: get_ij(x['e_x'], x['e_y']) if (np.all(pd.notnull(x['e_x'])) and np.all(pd.notnull(x['e_y']))) else -1,
        axis=1)
    del train_id['Source_loc'], train_id['end_loc']
    return train_id

In [ ]:
with open('./data/full_history', 'rb') as data_file:
    hi = pickle.load(data_file)
hi = process(hi)
hi

In [ ]:
with open('./data/full_history_plus', 'wb') as data_file:
    pickle.dump(hi, data_file)

In [ ]:
with open('./data/train-id4-crowd-grid3.txt', 'rb') as data_file:
    line = pickle.load(data_file)
with open('./data/test-id4-crowd-grid3.txt', 'rb') as data_file:
    test = pickle.load(data_file)

def get_grid_history(l,o,s,t,w,p,r,s,e):
    b = hi[(hi['Line']==l)&(hi['O_UP']==o)&(hi['source_id']==s)&(hi['target_id']==t)&(hi['is_workday']==w)&(hi['is_peek']==p)&(hi['is_rain']==r)&(hi['s_ij']==s)&(hi['e_ij']==e)]#
    if b.shape[0] > 0:
        return b.iloc[0]
    else :
        dic = {'Distance':[-1],'Diff_Time':[-1]}
        x = pd.DataFrame(dic)
        return x.iloc[0]
    
tmp = train.progress_apply(lambda x : get_grid_history(x['O_LINENO'],x['O_UP'],x['Source_Station_encode'],x['Target_Station_encode'],x['is_workday'],x['is_peek'],x['s_ij'],x['e_ij']), axis=1)
['grid_aver_diff'] = tmp['Diff_Time']
train['grid_aver_d'] = tmp['Distance']
train


tmp1 = test.progress_apply(lambda x : get_grid_history(x['O_LINENO'],x['O_UP'],x['Source_Station'],x['Target_Station'],x['is_workday'],x['is_peek'],x['s_ij'],x['e_ij']), axis=1)
test['grid_aver_diff'] = tmp1['Diff_Time']
test['grid_aver_d'] = tmp1['Distance']
test
with open('./data/test-id4-crowd-grid3.txt', 'wb') as data_file:
    pickle.dump(test, data_file)

# ID
拼接线路-上下行-站点编号

In [ ]:
def idfeature(a,b,c,d):
    if c < 10:
        c = '0'+str(c)
    if d < 10:
        d = '0'+str(d)
    return str(a)+str(b)+str(c)+str(d)
train['ID'] = train.apply(lambda x: idfeature(x['O_LINENO'],x['O_UP'],x['Source_Station'],x['Target_Station']),axis=1)
train = train[['ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'Distance',
       'O_TIME', 'aver_v', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'Diff_Time']]
test['ID'] = test.apply(lambda x: idfeature(x['O_LINENO'],x['O_UP'],x['Source_Station'],x['Target_Station']),axis=1)
test = test[['ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'Distance',
       'O_TIME', 'aver_v', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'Diff_Time']]

In [ ]:
with open('./data/train-id4-crowd-grid3.txt', 'wb') as data_file:
    pickle.dump(train, data_file)
with open('./data/test-id4-crowd-grid3.txt', 'wb') as data_file:
    pickle.dump(test, data_file)